In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.multitasking_lib import *

redis_conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)


In [36]:
######################
# Initiate Backtest  #
######################

def backtest_unitest(stocks):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    algo = ""
    fromDate = '2019-5-2'
    toDate = '2019-5-3'
    freq = '1min'
    backtest_msg={
            'stock':stocks,
            'sl':sl,
            'target':target,
            'qty':qty,
            'algo':algo,
            'fromDate':fromDate,
            'toDate':toDate,
            'freq':freq
        }

    for stock in stocks:
        redis_conn.set('stock',stock)
        # Step 3 : Store the OHLC and backtest data in the redis: to be used by plotter
        ohlc_dict = '[{"date":'+json.dumps(fromDate)+',"close":0,"high":0,"low":0,"open":0,"volume":0}]'
        redis_conn.set(stock,ohlc_dict)

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done',0)

    redis_conn.publish('kite_simulator',json.dumps(backtest_msg))

In [28]:
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(1)
loggerT.setLevel(21)
conn.set('done',1)
#TODO: Implement shared memory and split
backtest_manager = threadManager("backtest", ["kite_simulator","trade_handler"], [kite_simulator, trade_handler])
#trade_manager = threadManager("trade_manager", ["trade_handler"], [trade_handler])
order_manager = threadManager("order", ["order_handler"], [order_handler])

print(backtest_manager.job.is_alive())
print(order_manager.job.is_alive())
print(conn.pubsub_channels())
print(jobs)


True
True
['trade_handler', 'order_handler', 'kite_simulator']
[]


/code/lib/algo_lib.py:65: RuntimeWarning:

invalid value encountered in less

TRADE: B: EN:  : 2019-01-07 00:00 : WIPRO -> 243.8
TRADE: S: EN:  : 2019-01-09 00:00 : TCS -> 1886.95
TRADE: S: EX:  : 2019-02-14 00:00 : WIPRO -> 282.03
TRADE: S: EN:  : 2019-04-16 00:00 : WIPRO -> 280.9
ERROR:	Un-supported message: 166: ('1588428785674-0', {'msg': 'done'}) : <class 'json.decoder.JSONDecodeError'>
TRADE: B: EN:  : 2019-01-07 00:00 : WIPRO -> 243.8
TRADE: S: EX:  : 2019-04-16 00:00 : WIPRO -> 280.9
ERROR:	Un-supported message: 84: ('1588428826608-0', {'msg': 'done'}) : <class 'json.decoder.JSONDecodeError'>
TRADE: S: EN:  : 2019-05-02 09:39 : WIPRO -> 297.6
TRADE: B: EX:  : 2019-05-02 09:48 : WIPRO -> 297.95
TRADE: B: EN:  : 2019-05-02 10:33 : WIPRO -> 296.8
TRADE: S: EX:  : 2019-05-02 10:46 : WIPRO -> 296.8
TRADE: B: EN:  : 2019-05-02 11:54 : WIPRO -> 297.0
TRADE: S: EX:  : 2019-05-02 12:37 : WIPRO -> 294.8
TRADE: S: EN:  : 2019-05-02 12:52 : WIPRO -> 294.75
TRADE: B: EX:  : 2019-05-02 13:22

In [29]:
backtest_unitest(['WIPRO','TCS'])

In [37]:
backtest_unitest(['WIPRO'])

In [39]:
logMsg = redis_conn.get('logMsg')
print(logMsg)

Backtest Started: WIPRO :

S: EN:  : 2019-05-02 09:39 : WIPRO -> 297.6
B: EX:  : 2019-05-02 09:48 : WIPRO -> 297.95
B: EN:  : 2019-05-02 10:33 : WIPRO -> 296.8
S: EX:  : 2019-05-02 10:46 : WIPRO -> 296.8
B: EN:  : 2019-05-02 11:54 : WIPRO -> 297.0
S: EX:  : 2019-05-02 12:37 : WIPRO -> 294.8
S: EN:  : 2019-05-02 12:52 : WIPRO -> 294.75
B: EX:  : 2019-05-02 13:22 : WIPRO -> 294.4
S: EN:  : 2019-05-02 13:36 : WIPRO -> 294.65
B: EX:  : 2019-05-02 14:16 : WIPRO -> 293.95
B: EN:  : 2019-05-02 14:27 : WIPRO -> 293.9
S: EX:  : 2019-05-02 15:13 : WIPRO -> 293.8


In [45]:
# Terminate all the processes
backtest_manager.job.terminate()
order_manager.job.terminate()
time.sleep(0.1)
print(backtest_manager.job.is_alive())
print(order_manager.job.is_alive())
print(conn.pubsub_channels())
jobs

False
False
['trade_handler', 'order_handler', 'kite_simulator']


[]

In [26]:
#jobs[3].terminate()

In [27]:
jobs = []

In [43]:
from lib.charting_lib import *

def freedom_chart(symbol):
    ohlc_df = pd.read_json(redis_conn.get(symbol), orient='columns')
    ohlc_df.index.rename('date', inplace=True)
    
    trade_df = pd.read_json(redis_conn.get(symbol+'backtest'+'Trade'), orient='columns')
    print(trade_df)
    print(ohlc_df)
    return render_charts(ohlc_df, trade_df, symbol)

fig = freedom_chart('WIPRO')
fig

                       sell
2019-05-02 09:39:00  297.60
2019-05-02 09:48:00  297.95
2019-05-02 10:33:00  296.80
2019-05-02 10:46:00  296.80
2019-05-02 11:54:00  297.00
2019-05-02 12:37:00  294.80
2019-05-02 12:52:00  294.75
2019-05-02 13:22:00  294.40
2019-05-02 13:36:00  294.65
2019-05-02 14:16:00  293.95
2019-05-02 14:27:00  293.90
2019-05-02 15:13:00  293.80
                      close    high     low    open  volume
date                                                       
2019-05-02 09:15:00  298.00  298.65  297.50  298.55   30911
2019-05-02 09:16:00  297.00  298.00  297.00  298.00   30972
2019-05-02 09:17:00  296.50  297.10  296.00  297.00   85908
2019-05-02 09:18:00  296.50  296.80  296.25  296.40   28721
2019-05-02 09:19:00  296.70  296.95  296.40  296.50   21262
...                     ...     ...     ...     ...     ...
2019-05-02 15:25:00  294.00  294.00  293.45  293.50   29217
2019-05-02 15:26:00  293.95  294.00  293.90  294.00   42850
2019-05-02 15:27:00  294.00  294.00 

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Unit Test#1
freq='day'
exchange='NSE'
fromDate = '2019-01-01'
toDate = '2019-05-03'
stock='INFY'

state = 'INIT'
hash_key = stock+'_state'

try:
    all_keys = list(conn.hgetall(hash_key).keys())
    conn.hdel(hash_key,*all_keys)
except:
    pass

#conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)


In [ ]:
# Unit Test#2
stock = 'NDTV'
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

In [ ]:
conn.pubsub_channels()

In [ ]:
# Debug
stock='TCS'
#notification_despatcher(None, 'msg')
print(stock)
pd.read_json(conn.hget(stock+'_state','ohlc'))

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

## Scratchpad

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [7]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-2-2'
toDate  = '2019-2-28'
freq = 'day'
mydate = fromDate 
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [8]:
row = ohlc_data[stock].tail(1)

In [9]:
row

,close,high,low,open,volume
date,,,,,
2019-02-28,277.37,283.23,275.64,281.88,9428265


In [ ]:
import numpy as np
no = 1
for i in np.linspace(0,no-1,no):
    i = int(i)
    row = ohlc_data[stock].iloc[i]
    print(type(row))
    index = ohlc_data[stock].index[i]
    update_plot_cache(stock, row)
    print(index)
    print(ohlc_data[stock].iloc[i])
    # Construct Json message like Kite
    mydate = "{}-{}-{}".format(index.year,index.month,index.day)        
    msg = {exchange+":"+stock:{"ohlc":{'date':mydate,'open':row['open'],'high':row['high'],'low':row['low'],'close':row['close'],'volume':row['volume']}}}
    pdebug(msg)
    msg = json.dumps(msg)

In [ ]:
for i,r in ohlc_data[stock].iterrows():
    print(type(r))
    print(index)
    print(r)
    break

In [ ]:
#tmp_df = pd.DataFrame({'buy': ohlc_data.iloc[1:5]['close'], 'sell':ohlc_data.iloc[10:15]['close']})

tmp_df_s = pd.DataFrame()
tmp_df_b = pd.DataFrame()
tmp_df_b['buy'] = ohlc_data.iloc[-1:]['close']
tmp_df_s['sell'] = ohlc_data.iloc[20:25]['close']

tmp_df_b.append(tmp_df_s)

In [ ]:
msg = {exchange+":"+stock:{"ohlc":{'date':mydate,'open':row['open'],'high':row['high'],'low':row['low'],'close':row['close'],'volume':row['volume']}}}            